# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
csv_path = os.path.join("..", "WeatherPy", "Output", "output.csv")
original_weather_df = pd.read_csv(csv_path)
original_weather_df[["Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"]] = original_weather_df[["Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Date"]].astype(float)
original_weather_df.dropna(axis=0,how='all', inplace=True)#drop all NaN items from data frame
original_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hithadhoo,-0.60,73.08,81.95,68.0,1.0,8.46,MV,1.608105e+09
2,Punta Arenas,-53.15,-70.92,48.20,61.0,0.0,44.96,CL,1.608105e+09
3,Lipari,38.47,14.95,54.00,93.0,75.0,2.55,IT,1.608105e+09
4,Taltal,-25.40,-70.48,59.65,79.0,62.0,1.12,CL,1.608105e+09
5,Port Alfred,-33.59,26.89,73.00,72.0,10.0,5.01,ZA,1.608105e+09
...,...,...,...,...,...,...,...,...,...
644,Leshukonskoye,64.90,45.76,12.42,96.0,86.0,5.61,RU,1.608105e+09
645,Westport,41.14,-73.36,26.60,42.0,1.0,8.05,US,1.608105e+09
646,Birao,10.28,22.79,83.84,14.0,37.0,11.32,CF,1.608105e+09
647,São João da Barra,-21.64,-41.05,73.76,87.0,100.0,10.07,BR,1.608105e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [30]:
#configure gmaps using my Google API key
gmaps.configure(api_key=g_key)

coordinates = original_weather_df[['Lat', 'Lng']].astype(float)
humidity = original_weather_df["Humidity"].astype(float)
# Customize the size of the figure
figure_layout = {
    'width': '533px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1.5)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [24]:
good_weather_df = original_weather_df.copy()

good_weather_df = good_weather_df[(good_weather_df['Max Temp'] > 70) & (good_weather_df['Max Temp'] < 80)] #only show cities with a max temperature between 70 and 80 degrees
good_weather_df = good_weather_df[good_weather_df['Wind Speed'] > 10] #and only show cities where the wind speed is <10 mph
good_weather_df = good_weather_df[good_weather_df['Cloudiness'] == 0] #and only show cities where there's no clouds
good_weather_df.dropna(axis=0,how='any', inplace=True) #removing any of the cities that have null values

#good_weather_df
original_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hithadhoo,-0.60,73.08,81.95,68.0,1.0,8.46,MV,1.608105e+09
2,Punta Arenas,-53.15,-70.92,48.20,61.0,0.0,44.96,CL,1.608105e+09
3,Lipari,38.47,14.95,54.00,93.0,75.0,2.55,IT,1.608105e+09
4,Taltal,-25.40,-70.48,59.65,79.0,62.0,1.12,CL,1.608105e+09
5,Port Alfred,-33.59,26.89,73.00,72.0,10.0,5.01,ZA,1.608105e+09
...,...,...,...,...,...,...,...,...,...
644,Leshukonskoye,64.90,45.76,12.42,96.0,86.0,5.61,RU,1.608105e+09
645,Westport,41.14,-73.36,26.60,42.0,1.0,8.05,US,1.608105e+09
646,Birao,10.28,22.79,83.84,14.0,37.0,11.32,CF,1.608105e+09
647,São João da Barra,-21.64,-41.05,73.76,87.0,100.0,10.07,BR,1.608105e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [25]:
hotel_df  = good_weather_df.copy()
hotel_df["Hotel Name"] = "NaN" #initialize Hotel Name column
hotel_df

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# find the closest hotel to coordinates. use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    
    cityName = row['City'] #get city name type from df
    lat = row['Lat'] #get latitude from df
    lng = row['Lng'] #get longitude from df
    
    print(lat)
    print(lng)
    
    params = {
    "location": f"{lat},{lng}", #substituting latitude and longitude into the API call
    "rankby": "distance",
    "type": "restaurant",
    "radius": 5000, #within 5000 meters of this location
    "key": g_key,
    }    

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {cityName}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    print(results)
    
#     try:
#         print(f"Closest {cityName} restaurant is {results[0]['name']}.")
        
# #         types_df.loc[index, 'name'] = results[0]['name']
# #         types_df.loc[index, 'address'] = results[0]['vicinity']
# #         types_df.loc[index, 'price_level'] = results[0]['price_level']
# #         types_df.loc[index, 'rating'] = results[0]['rating']
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
#     print("------------")

-33.93
18.42
Retrieving Results for Index 8: Cape Town.
[]
-28.32
27.62
Retrieving Results for Index 40: Senekal.
[]
11.68
10.19
Retrieving Results for Index 47: Azare.
[]
-24.87
113.63
Retrieving Results for Index 78: Carnarvon.
[]
25.02
37.27
Retrieving Results for Index 170: Umluj.
[]
18.44
1.41
Retrieving Results for Index 201: Kidal.
[]
21.46
-71.14
Retrieving Results for Index 274: Cockburn Town.
[]
-28.77
114.6
Retrieving Results for Index 275: Geraldton.
[]
-33.02
27.91
Retrieving Results for Index 277: East London.
[]
28.97
50.84
Retrieving Results for Index 295: Bushehr.
[]
12.97
-0.14
Retrieving Results for Index 314: Bogandé.
[]
-29.66
17.89
Retrieving Results for Index 535: Springbok.
[]
3.48
34.15
Retrieving Results for Index 540: Kaabong.
[]
-32.5
137.77
Retrieving Results for Index 547: Port Augusta.
[]


In [26]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
# Add marker layer ontop of heat map


# Display figure

# fig = gmaps.figure(layout=figure_layout)  # Assign the marker layer to a variable
# markers = gmaps.marker_layer(coordinates)
# # Add the layer to the map
# fig.add_layer(markers)

# heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 0.05)
# fig.add_layer(heat_layer)
# fig
